<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/ReflectiveUpdater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Callable, Iterable, Optional, Protocol, Tuple, List
import logging


# --- Protocols / lightweight contracts --------------------------------------

class EpisodeLike(Protocol):
    @property
    def input(self) -> Any: ...
    @property
    def outcome(self) -> Any: ...


class MemoryLike(Protocol):
    def recent(self) -> Iterable[EpisodeLike]: ...


class WorldModelLike(Protocol):
    def simulate(self, x: Any) -> Any: ...
    def update(self, contradictory: List[EpisodeLike]) -> None: ...


class SelfModelLike(Protocol):
    def adapt(self, contradictory: List[EpisodeLike]) -> None: ...


SimilarityFn = Callable[[Any, Any], float]


# --- Implementation -----------------------------------------------------------

class ReflectiveUpdater:
    """
    Cross-check recent episodes against the world model; on contradiction,
    update both the world and self models.

    Parameters
    ----------
    episodic_memory : MemoryLike
        Provides recent() -> iterable of episodes with .input and .outcome.
    world_model : WorldModelLike
        Must implement simulate(x) and update(episodes).
    self_model : SelfModelLike
        Must implement adapt(episodes).
    similarity_fn : Callable[[pred, actual], float]
        Returns a consistency score; higher means more similar.
    threshold : float, default 0.8
        Minimum score to consider pred and actual consistent (inclusive).
    logger : logging.Logger, optional
        For warnings/debugging; defaults to module logger.
    """

    def __init__(
        self,
        episodic_memory: MemoryLike,
        world_model: WorldModelLike,
        self_model: SelfModelLike,
        similarity_fn: SimilarityFn,
        *,
        threshold: float = 0.8,
        logger: Optional[logging.Logger] = None,
    ) -> None:
        self.memory = episodic_memory
        self.world = world_model
        self.self_model = self_model
        self.similarity_fn = similarity_fn
        self.threshold = float(threshold)
        self.log = logger or logging.getLogger(__name__)

    # --- Core loop -----------------------------------------------------------

    def detect_contradictions(self) -> Iterable[EpisodeLike]:
        """
        Yield episodes for which world.simulate(input) is not consistent
        with episode.outcome per the configured similarity/threshold.

        Errors from simulate or similarity are treated as contradictions
        and logged, so learning can still proceed.
        """
        for episode in self.memory.recent():
            try:
                prediction = self.world.simulate(episode.input)
            except Exception as e:
                self.log.warning("simulate() failed; treating as contradiction: %s", e)
                yield episode
                continue

            try:
                if not self._is_consistent(prediction, episode.outcome):
                    yield episode
            except Exception as e:
                self.log.warning("similarity check failed; treating as contradiction: %s", e)
                yield episode

    def revise_model(self) -> int:
        """
        Collect contradictions and update world/self models.
        Returns the number of contradictory episodes processed.
        """
        contradictory = list(self.detect_contradictions())
        if not contradictory:
            self.log.debug("No contradictions detected.")
            return 0

        self.log.debug("Updating with %d contradictory episodes.", len(contradictory))
        self.world.update(contradictory)
        self.self_model.adapt(contradictory)
        return len(contradictory)

    # --- Consistency utilities ----------------------------------------------

    def consistency_score(self, pred: Any, actual: Any) -> float:
        """Return similarity score between predicted and actual outcomes."""
        score = float(self.similarity_fn(pred, actual))
        return score

    def _is_consistent(self, pred: Any, actual: Any) -> bool:
        """Inclusive threshold check: score >= threshold is consistent."""
        return self.consistency_score(pred, actual) >= self.threshold